In [1]:
import csv, sqlite3

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

In [2]:
import pandas as pd

In [3]:
%load_ext sql

In [4]:
%sql sqlite:///RealWorldData.db

In [17]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")


566

### Problem 1: Find the total number of crimes recorded in the CRIME table.

In [142]:
%sql select count(*) \
    from CHICAGO_CRIME_DATA

 * sqlite:///RealWorldData.db
Done.


count(*)
533


### Problem 2: List community areas with per capita income less than 11000.

In [24]:
%sql select COMMUNITY_AREA_NAME, PER_CAPITA_INCOME \
    from CENSUS_DATA \
    where PER_CAPITA_INCOME < 11000

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


### Problem 3: List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [63]:
%sql select CASE_NUMBER  \
    from CHICAGO_CRIME_DATA \
    where DESCRIPTION like '%MINOR%'

 * sqlite:///RealWorldData.db
Done.


CASE_NUMBER
HL266884
HK238408


### Problem 4: List all kidnapping crimes involving a child?

In [75]:
%sql select *  \
    from CHICAGO_CRIME_DATA \
    WHERE PRIMARY_TYPE like '%KIDNAP%' and DESCRIPTION like '%CHILD%'

 * sqlite:///RealWorldData.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem 5: What kinds of crimes were recorded at schools?

In [73]:
%sql select Distinct(PRIMARY_TYPE)  \
    from CHICAGO_CRIME_DATA \
    where LOCATION_DESCRIPTION like '%SCHOOL%'

 * sqlite:///RealWorldData.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6: List the average safety score for each type of school.

In [108]:
%sql SELECT 'Elementary, Middle, or High School' as SCHOOL_TYPE , AVG(SAFETY_SCORE) as avg_safety_score \
    FROM CHICAGO_PUBLIC_SCHOOLS_DATA \
    GROUP BY SCHOOL_TYPE;


 * sqlite:///RealWorldData.db
Done.


SCHOOL_TYPE,avg_safety_score
"Elementary, Middle, or High School",49.50487329434698


### Problem 7: List 5 community areas with highest % of households below poverty line

In [111]:
%sql select COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY \
    from CENSUS_DATA \
    ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC \
    LIMIT 5
    

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8: Which community area is most crime prone?

In [128]:
%sql select COMMUNITY_AREA_NUMBER , COUNT(PRIMARY_TYPE)  \
    from CHICAGO_CRIME_DATA \
    GROUP BY COMMUNITY_AREA_NUMBER \
    ORDER BY COUNT(PRIMARY_TYPE) DESC \
    LIMIT 1

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NUMBER,COUNT(PRIMARY_TYPE)
25.0,43


### Problem 9: Use a sub-query to find the name of the community area with highest hardship index

In [137]:
%sql select COMMUNITY_AREA_NAME, HARDSHIP_INDEX \
    from CENSUS_DATA \
    where HARDSHIP_INDEX= (select MAX(HARDSHIP_INDEX) from CENSUS_DATA )

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


### Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [139]:
%sql select CD.COMMUNITY_AREA_NAME, CCD.COMMUNITY_AREA_NUMBER, COUNT(CCD.PRIMARY_TYPE) as Crime_number \
    from CENSUS_DATA as CD, CHICAGO_CRIME_DATA as CCD \
    where CD.COMMUNITY_AREA_NUMBER= CCD.COMMUNITY_AREA_NUMBER \
    group by CD.COMMUNITY_AREA_NAME, CCD.COMMUNITY_AREA_NUMBER \
    order by Crime_number DESC \
    LIMIT 1

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,Crime_number
Austin,25.0,43
